# Enhancing Accuracy in Healthcare Record Interpretation with AI21 Models and Amazon SageMaker

In the context of healthcare, the veracity of information is paramount. Accurate data is crucial for informed decision-making, enabling healthcare professionals to deliver quality care, reduce misdiagnoses, and prevent adverse events. This notebook shows how to use AI21's [Contextual Answers](https://aws.amazon.com/marketplace/pp/prodview-gwbjdp3tmh3bw) model to derive grounded answers from healthcare documents. In this notebook, we focus on answering questions from patient healthcare records.

## Install and import dependencies

In [ ]:
!pip install -U "ai21[AWS]>=2.0.0"
#!pip install sagemaker
#!pip install boto3

In [28]:
import json
from sagemaker import ModelPackage
from sagemaker import get_execution_role
import sagemaker as sage
import boto3
from ai21 import AI21SageMakerClient
import ai21
boto_session = boto3.Session()
region = boto3.Session().region_name

## Get AI21 Model Package
Obtain the updated ARN (Amazon Resource Name) for the model package "contextual-answers" using the ai21 SageMaker library, and acquire the required role and session to interact with Amazon SageMaker.

In [29]:
model_package_arn = ai21.SageMaker.get_model_package_arn(model_name="contextual-answers", region=region)
role = get_execution_role()
sagemaker_session = sage.Session()

## Deploy SageMaker Endpoint
Set up a real-time inference endpoint named "contextual-answers" with a specified content type of "application/json". You can choose from a few different instances to use; the `ml.p4d.24xlarge` will be most performant, but also costly.

(Note that the model takes between 5 to 7 minutes to deploy)

In [30]:
endpoint_name = "contextual-answers"

content_type = "application/json"

real_time_inference_instance_type = (
#    "ml.p4d.24xlarge"    # Recommended instance
#     "ml.g5.48xlarge"    # Cheaper and faster - recommended for relatively short inputs/outputs
#      "ml.g5.12xlarge"    # Even Cheaper and faster - up to 10K characters
      "ml.g4dn.12xlarge"  #cheapest instance  
)

# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type,
    endpoint_name=endpoint_name, 
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=600
)

-------------!

# Ask Healthcare Questions Using Contextual Answers

Next, we will use Contextual Answers to address inquiries related to this specific patient's health records. Contextual Answers provides a platform that enables us to extract precise information from unstructured data by as natural language questions. Importantly, Contextual Answers will not only answer the questions provided, it will refuse to asnwer questions whose answers are not in the document. By strictly adhering to the text of the document, contextual answers provides increased safety and reliability.

In [31]:
patient_record_context='''
# Chief Complaint
No complaints.

# History of Present Illness
Timothy142 is a 6 year-old non-hispanic black male.

# Social History
 Patient has never smoked.

Patient comes from a middle socioeconomic background. Patient currently has Humana.

# Allergies
No Known Allergies.

# Medications
No Active Medications.

# Assessment and Plan


## Plan
Patient was given the following immunizations: influenza, seasonal, injectable, preservative free. 
The following procedures were conducted:
- medication reconciliation (procedure)
'''

### (Optional) Read from data in S3
In complex applications, healthcare data will frequently be stored in data lakes such as S3 object store. In this case, you can read the data from there prior to passing it to AI21's Contextual Answers model.




In [32]:
import boto3

def read_s3_object(s3_url):
    if s3_url.startswith('s3://'):
        s3_url = s3_url[5:]
        bucket_name, object_key = s3_url.split('/', 1)
    else:
        raise ValueError("URL must start with s3://")
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    return data

#s3_url = '' #in the format s3://BUCKET/directory/file.txt
#data = read_s3_object(s3_url)
#context=data.decode('utf-8')

## Answer Questions whose answers are in the document
First we will provide questions that can be directly answered from the document.

If the first invokation of the endpoint fails, **just simply rerun the cell**; the first invocation of the endpoint sometimes has an error.

In [33]:
questions=["How old is the patient?",
           "What insurance does the patient have?", 
           "What medical procedures were performed?"]

In [36]:
client = AI21SageMakerClient(
    endpoint_name=endpoint_name,
)
for question in questions:
    response = client.answer.create(
      session=boto_session,
      context=patient_record_context,
      question=question,
)
    print(response.answer)
        

Timothy142 is a 6 year-old non-hispanic black male.
The patient has Humana
The following procedures were conducted:
- medication reconciliation (procedure)


In [37]:
questions_not_in_doc=[
    "What are insurances similar to Humana?",
    "Does the patient's family have a history of heard disease?",
    "Is Timothy competent to choose his own medication?"]

In [38]:
for question in questions_not_in_doc:
    response = client.answer.create(
      session=boto_session,
      context=patient_record_context,
      question=question,
     )
    answer=response.answer
    if response.answer is None:
        answer="Answer not in context"
    print(answer)
        

Answer not in context
Answer not in context
Answer not in context


Note that Contextual Answers refuses to answer questions that are not in the document; even when there is a possiblity to deduce the answer based on facts outside of the document. For example, given that Timothy is only 6 years old, it is reasonable to assume that he is not competent to choose his own medication. Nevertheless, **because this fact is not explicity said in the document** Contextual Answers correctly does not return any answers. This strict grounding in the document text is desirable in many regulated industries, such as healthcare.

## Clean up resources
Lastly, we will clean up all created resources, specifically the SageMaker Endpoint


In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)
model.delete_model()